# Create Dataset
Aim to create the dataset of abstract for task classification.
From OpenAlex inverted_abstract will create a reconstructed string version of the abstract and classify this task in regard of the dataset the paper references

# Using list of paper that references a dataset with OpenAlex

In [1]:
import pandas as pd
import requests
import numpy as np
import csv

datasets_context = {}
ds_reader = csv.DictReader(open('../../data/datasets.csv'))
for ds in ds_reader:
    datasets_context[ds["name"]] = ds["context"]
    
df = pd.read_csv("../../results/extracted_csv/paper_openalex.csv")

In [3]:
query_url = "https://api.openalex.org/works/doi"

paper_abstract = {}
with open("../../results/extracted_csv/paper_openalex.csv","r") as paper_csv:
    with open("../../data/abstract_dataset.csv","w") as abstract_file:
        abstract_file.write("title,doi,abstract,task")
        text = csv.reader(paper_csv,)
        header=next(text)
        for i,l in enumerate(text):
            ds = datasets_context[l[-1]]
            if not paper_abstract.get(l[1],None):
                r = requests.get(query_url+l[1])
                if r.status_code == 200:
                    r_json = r.json()
                    if r_json["abstract_inverted_index"]:
                        abstract = np.full(1000,"",dtype=object)
                        for w in r_json["abstract_inverted_index"]:
                            for indices in r_json["abstract_inverted_index"][w]:
                                abstract[indices] = ''.join(filter(str.isalnum, w)).lower()
                        abstract = abstract[abstract != ""]
                        str_abstract = ' '.join(abstract)
                        paper_abstract[l[1]] = (str_abstract,ds)
                        abstract_file.write(f'\n{l[0]},{l[1]},"{paper_abstract[l[1]][0]}",{ds}')
            elif ds != paper_abstract[l[1]][1]:
                abstract_file.write(f'\n{l[0]},{l[1]},"{paper_abstract[l[1]][0]}",{ds}')

## Using list of papers that cite a dataset or an organ in the abstract

In [2]:
keyword_dict = {
    'acdc':'Cardiac',
    'mscmr':"Cardiac",
    'cardiac':'Cardiac',
    'brats':"Brain",
    'brain':"Brain",
    'i2cvb':'Prostate',
    'prostatex':'Prostate',
    'promise12':'Prostate',
    'prostate':'Prostate'
}

In [13]:
keyword_dict = {}
with open("../../data/keywords.csv",'r') as f:
    for l in csv.DictReader(f):
        keyword_dict[l["keyword"]] = l["organ"]

In [14]:
keyword_dict

{'acdc': 'Cardiac',
 'mscmr': 'Cardiac',
 'cardiac': 'Cardiac',
 'brats': 'Brain',
 'brain': 'Brain',
 'i2cvb': 'Prostate',
 'prostatex': 'Prostate',
 'promise12': 'Prostate',
 'prostate': 'Prostate',
 'lung': 'Lung',
 'lungs': 'Lung'}

In [15]:
query_url = "https://api.openalex.org/works"
with open('../../results/classification/raw_abstract.csv',"w") as test_file:
    test_file.write(f'title,doi,publication_year,abstract,task')
    for year in range(2023,2024):
        print(year)
        next_page = True
        page_number = 1
        while next_page:
            query_param = {
                'page':page_number,
                'filter':f"concepts.id:C89600930,concepts.id:C154945302,has_abstract:true,publication_year:{year}"
            }
            r_year = requests.get(query_url,params=query_param)
            if r_year.status_code == 200:
                r_json = r_year.json()
                if not r_json["results"]:
                    next_page = False
                else:
                    for paper in r_json["results"]:
                        abstract = np.full(2500,"",dtype=object)
                        for w in paper["abstract_inverted_index"]:
                            for indices in paper["abstract_inverted_index"][w]:
                                if indices < 2500:
                                    abstract[indices] = ''.join(filter(str.isalnum, w)).lower()
                        abstract = abstract[abstract != ""]
                        str_abstract = ' '.join(abstract)
                        title = paper["title"]
                        if title:
                            title = title.replace(",","")
                            title = title.replace("\n","")
                        for k in keyword_dict:
                            if k in str_abstract:
                                test_file.write(f'\n{title},{paper["doi"]},{paper["publication_year"]},"{str_abstract}",{keyword_dict[k]}')
                    page_number += 1
            else:
                next_page=False

2023


In [4]:
import pandas as pd
df = pd.read_csv("../../results/classification/raw_abstract.csv")
df = df.drop_duplicates(subset=['title','task'])
df.to_csv("../../data/abstract_dataset_clean2.csv")

## List of non-relevant papers selected manually to add to the dataset to make the model able to detect non related papers

In [6]:
query_url = "https://api.openalex.org/works/doi"

with open("../../data/paper_other.csv","r") as paper_csv:
    with open("../../data/abstract_dataset_nonrelevant.csv","w") as abstract_file:
        abstract_file.write("title,doi,publication_year,abstract,task")
        text = csv.reader(paper_csv,)
        for i,l in enumerate(text):
            r = requests.get(query_url+l[0])
            if r.status_code == 200:
                r_json = r.json()
                if r_json["abstract_inverted_index"]:
                    abstract = np.full(1000,"",dtype=object)
                    for w in r_json["abstract_inverted_index"]:
                        for indices in r_json["abstract_inverted_index"][w]:
                            abstract[indices] = ''.join(filter(str.isalnum, w)).lower()
                    abstract = abstract[abstract != ""]
                    str_abstract = ' '.join(abstract)
                    title = r_json["title"]
                    title = title.replace(",","")
                    title = title.replace("\n","")
                    abstract_file.write(f'\n{title},{l[0]},"{str_abstract}",Other')